In [4]:
import pprint
import nltk
from nltk.corpus import stopwords
from gensim import corpora

In [5]:
text_corpus = [
    "Human machine interface for lab abc computer and machine applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [7]:
stop_words = stopwords.words('english')
stop_words = stop_words + ['com', 'edu', 'subject', 'lines', 'organization', 'would', 'article', 'could']

In [8]:
# Create a set of frequent words
#stop_words = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stop_words]
         for document in text_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
pprint.pprint(processed_corpus)

[['human', 'machine', 'interface', 'computer', 'machine'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [9]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
# train word2vec on the two sentences
model = gensim.models.Word2Vec(processed_corpus, min_count=1)

2021-02-08 17:52:56,517 : INFO : collecting all words and their counts
2021-02-08 17:52:56,518 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-08 17:52:56,518 : INFO : collected 13 word types from a corpus of 31 raw words and 9 sentences
2021-02-08 17:52:56,519 : INFO : Loading a fresh vocabulary
2021-02-08 17:52:56,520 : INFO : effective_min_count=1 retains 13 unique words (100% of original 13, drops 0)
2021-02-08 17:52:56,521 : INFO : effective_min_count=1 leaves 31 word corpus (100% of original 31, drops 0)
2021-02-08 17:52:56,522 : INFO : deleting the raw counts dictionary of 13 items
2021-02-08 17:52:56,522 : INFO : sample=0.001 downsamples 13 most-common words
2021-02-08 17:52:56,523 : INFO : downsampling leaves estimated 3 word corpus (12.6% of prior 31)
2021-02-08 17:52:56,523 : INFO : estimated required memory for 13 words and 100 dimensions: 16900 bytes
2021-02-08 17:52:56,524 : INFO : resetting layer weights
2021-02-08 17:52:56,530 : INFO :

In [20]:
#model2 = gensim.models.Word2Vec(iter=1)  # an empty model, no training yet
#model2.build_vocab(processed_corpus)  # can be a non-repeatable, 1-pass generator
new_doc = "Human computer interaction"
model.train(new_doc.lower().split(),total_examples=1,epochs=5)  # can be a non-repeatable, 1-pass generator

2021-02-08 18:02:00,455 : INFO : training model with 3 workers on 13 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2021-02-08 18:02:00,458 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-02-08 18:02:00,458 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-02-08 18:02:00,459 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-08 18:02:00,460 : INFO : EPOCH - 1 : training on 24 raw words (0 effective words) took 0.0s, 0 effective words/s
2021-02-08 18:02:00,460 : WARNING : EPOCH - 1 : supplied example count (3) did not equal expected count (1)
2021-02-08 18:02:00,463 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-02-08 18:02:00,464 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-02-08 18:02:00,465 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-02-08 18:02:00,465 : INFO : EPOCH - 2 : training on 24 raw words (0 effec

(0, 120)

In [21]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

2021-02-08 18:02:58,018 : INFO : Creating /Users/imayak/gensim-data


[=================================================-] 100.0% 1662.2/1662.8MB downloaded


2021-02-08 18:06:16,913 : INFO : word2vec-google-news-300 downloaded
2021-02-08 18:06:16,916 : INFO : loading projection weights from /Users/imayak/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2021-02-08 18:07:00,943 : INFO : loaded (3000000, 300) matrix from /Users/imayak/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [25]:
for index, word in enumerate(wv.index2word):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index2word)} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


In [28]:
vec_king = wv['king']
vec_king.shape

(300,)

In [29]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


In [34]:
print(wv.most_similar(positive=['video', 'conference'], topn=10))

[('Efforting_soundbites', 0.6146546602249146), ('Watch_TimesCast_daily', 0.5961092114448547), ('teleconference', 0.5916281342506409), ('INCLUDES_previously_unreleased', 0.568611741065979), ('video_clips', 0.540194034576416), ('videos', 0.5204994678497314), ('www.bigtennetwork.com', 0.5163251757621765), ('audio', 0.5104407072067261), ('mms_:/', 0.5060652494430542), ('videotape', 0.5015642642974854)]


In [35]:
print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

car


/Users/imayak/exper/python_nlp_venv/lib/python3.8/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [37]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

2021-02-08 18:14:45,560 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-02-08 18:14:45,561 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [38]:
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

2021-02-08 18:14:56,059 : INFO : collecting all words and their counts
2021-02-08 18:14:56,064 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-08 18:14:56,158 : INFO : collected 6981 word types from a corpus of 58152 raw words and 300 sentences
2021-02-08 18:14:56,159 : INFO : Loading a fresh vocabulary
2021-02-08 18:14:56,165 : INFO : effective_min_count=5 retains 1750 unique words (25% of original 6981, drops 5231)
2021-02-08 18:14:56,166 : INFO : effective_min_count=5 leaves 49335 word corpus (84% of original 58152, drops 8817)
2021-02-08 18:14:56,172 : INFO : deleting the raw counts dictionary of 6981 items
2021-02-08 18:14:56,173 : INFO : sample=0.001 downsamples 51 most-common words
2021-02-08 18:14:56,173 : INFO : downsampling leaves estimated 35935 word corpus (72.8% of prior 49335)
2021-02-08 18:14:56,179 : INFO : estimated required memory for 1750 words and 100 dimensions: 2275000 bytes
2021-02-08 18:14:56,180 : INFO : resetting layer weight

In [47]:
import tempfile

with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    model.save(temporary_filepath)
    #
    # The model is now safely stored in the filepath.
    # You can copy it to other machines, share it with others, etc.
    #
    # To load a saved model:
    #
    new_model = gensim.models.Word2Vec.load(temporary_filepath)

2021-02-08 19:27:51,407 : INFO : saving Word2Vec object under /var/folders/58/6ywxrmgj7mv2t7lhy0t2h7bh00r92n/T/gensim-model-j2lysxmm, separately None
2021-02-08 19:27:51,408 : INFO : not storing attribute vectors_norm
2021-02-08 19:27:51,409 : INFO : not storing attribute cum_table
2021-02-08 19:27:51,422 : INFO : saved /var/folders/58/6ywxrmgj7mv2t7lhy0t2h7bh00r92n/T/gensim-model-j2lysxmm
2021-02-08 19:27:51,423 : INFO : loading Word2Vec object from /var/folders/58/6ywxrmgj7mv2t7lhy0t2h7bh00r92n/T/gensim-model-j2lysxmm
2021-02-08 19:27:51,440 : INFO : loading wv recursively from /var/folders/58/6ywxrmgj7mv2t7lhy0t2h7bh00r92n/T/gensim-model-j2lysxmm.wv.* with mmap=None
2021-02-08 19:27:51,440 : INFO : setting ignored attribute vectors_norm to None
2021-02-08 19:27:51,441 : INFO : loading vocabulary recursively from /var/folders/58/6ywxrmgj7mv2t7lhy0t2h7bh00r92n/T/gensim-model-j2lysxmm.vocabulary.* with mmap=None
2021-02-08 19:27:51,442 : INFO : loading trainables recursively from /var/

In [48]:
more_sentences = [
    ['Advanced', 'users', 'can', 'load', 'a', 'model',
     'and', 'continue', 'training', 'it', 'with', 'more', 'sentences'],
]
new_model.build_vocab(more_sentences, update=True)
new_model.train(more_sentences, total_examples=model.corpus_count, epochs=model.epochs)

# cleaning up temporary file
import os
os.remove(temporary_filepath)

2021-02-08 19:28:07,747 : INFO : collecting all words and their counts
2021-02-08 19:28:07,750 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-08 19:28:07,750 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
2021-02-08 19:28:07,751 : INFO : Updating model with new vocabulary
2021-02-08 19:28:07,752 : INFO : New added 0 unique words (0% of original 13) and increased the count of 0 pre-existing words (0% of original 13)
2021-02-08 19:28:07,752 : INFO : deleting the raw counts dictionary of 13 items
2021-02-08 19:28:07,753 : INFO : sample=0.001 downsamples 0 most-common words
2021-02-08 19:28:07,753 : INFO : downsampling leaves estimated 0 word corpus (0.0% of prior 0)
2021-02-08 19:28:07,756 : INFO : estimated required memory for 1750 words and 100 dimensions: 2275000 bytes
2021-02-08 19:28:07,757 : INFO : updating layer weights
2021-02-08 19:28:07,758 : WARNING : Effective 'alpha' higher than previous training cycles
2021-

In [52]:
new_model.most_similar(positive=['training'], topn=10)

<ipython-input-52-b4d9c6893336>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  new_model.most_similar(positive=['training'], topn=10)


[('at', 0.999830961227417),
 ('minister', 0.9998297691345215),
 ('were', 0.9998275637626648),
 ('out', 0.9998210668563843),
 ('which', 0.9998193383216858),
 ('other', 0.9998165369033813),
 ('or', 0.9998159408569336),
 ('those', 0.9998151063919067),
 ('several', 0.9998124837875366),
 ('when', 0.9998108744621277)]

In [54]:
!pip install sklearn


     |████████████████████████████████| 7.2 MB 5.6 MB/s eta 0:00:01
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn ... done


In [56]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(new_model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

ModuleNotFoundError: No module named 'plotly'